In [14]:
import hashlib
import datetime
import time
import json

# INITIAL_BITS = 0x1d777777
INITIAL_BITS = 0x1e777777
MAX_32BIT = 0xffffffff

In [15]:
# ここのブロックを定義するクラス
class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    # 特殊メソッドの__setitem__を利用し、setattrメソッドで属性を追加
    def __setitem__(self, key, value):
        # pythonでは.(ドット)を使うことで直接オブジェクトにアクセスすることができるが、setattrを利用すると、どのオブジェクトにどのようなkeyとvalueをつけるか指定できる
        setattr(self, key, value)

    def to_json(self):
        return {
            "index": self.index,
            "prev_hash": self.prev_hash,
            "stored_data": self.data,
            "timestamp": self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            # rjust(8, "0")は全部で指定された文字になるように右寄せにするメソッド。この場合、全部で８文字になるように右寄せし、左側の足りない部分は０で埋める処理をする。
            "bits" : hex(self.bits)[2:].rjust(8, "0"),
            "nonce" : hex(self.nonce)[2:],
            "elapsed_time": self.elapsed_time,
            "block_hash": self.block_hash
        }

    # ブロックヘッダを構築したのち、それをSHA256でハッシュ化し、その結果を返す処理
    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + str(self.bits)[2:] + str(self.nonce)# [2:]で先頭２文字を飛ばしているのは、16真数を表す0xが先頭についているため。0xを飛ばしてbitsの値そのものを抜き出すためにソートをしている。
        # hashlibは文字列をエンコードしたものを引数として渡す必要があるため、上の行のようみ全てを文字列に変換したものを.encode()によってエンコードしている。
        h = hashlib.sha256(blockheader.encode()).hexdigest()# .encode()は引数を設定しなければでジョルとでUTF-8が使われる。
        # 冒頭で定義したブロックハッシュの変数に上の行で計算したハッシュ値を代入
        self.block_hash = h
        return h

    # 与えられたbitsからtargetを算出
    def calc_target(self):
        # bitsを右に24ビットシフトさせ、3を引いている。
        # Bitsは1e777777といった形で与えられているので右に２４ビットずらすと、exponentに当たる1eだけが残り、そこからcoefficientが3バイトのため後々桁数をずらす計算をする際に生合成を取るために３を引く。
        exponent_bytes = (self.bits >> 24) - 3
        # 1バイトが８ビットのため、８を掛けている
        exponent_bits = exponent_bytes * 8
        # bitsからcoefficientを抽出するために行う。
        # &はビット論理積で両方が１の時のみ1になり、それ以外は0となるため、ビット論理積をとれば相手側が１の部分のみが残って、桁数が同じであれば同じ結果になる。
        # 加えて、桁数が6桁で、bitsが8桁のため先頭2桁の1eを排除することができ、結果としてcoefficientの部分のみが抽出できる。
        coefficient = self.bits & 0xffffff
        # coefficientをexponent_bits分、左にシフトさせることでターゲットが算出され、返り値として返される。
        return coefficient << exponent_bits

    # 計算されたハッシュ値が先ほど計算したターゲットよりも小さいかどうかを判定している。
    def check_valid_hash(self):
        # 左側の「~~16)」はハッシュ値を１６真数であると見立てて１０進数の整数に変換するための記述で、これによってハッシュ値とターゲットを比較できるようになる。
        return int(self.calc_blockhash(), 16) <= self.calc_target()

In [16]:
# ブロックの関係性を定義するクラス
class BlockChain():
    def __init__(self, initial_bits):
        # 変数としてブロックの情報を格納するからの配列
        self.chain = []
        # 難易度を定義するbits
        self.initial_bits = initial_bits

    # chainの配列にブロックのデータを追加するメソッド
    def add_block(self, block):
        self.chain.append(block)

    # chainの配列のその時点での最後の要素を取り出し、json形式で出力する
    def getblockinfo(self, index=-1):
        # json.dumpsは辞書型をJSON形式に変換するメソッド。PythonではJSON形式は文字列型として扱われる。
        # 第１引数は変換したい辞書型のデータを指定し、
        # 第２引数ではインデントの大きさ、
        # 第３引数では辞書の出力がキーでソートできるようにTrueを指定、
        # 第４引数ensure_ascii=Falseは日本語表記出力結果がひと続きで出力されるので、可読性を高めるためにいJSON形式で出力する。
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))

    # ブロックをつなげていくための処理
    def mining(self, block):
        # 処理がスタートした時間を取り出している。
        start_time = int(time.time() * 1000)
        # 条件が一致している間はそこから続くfor文を伊h他すら続けていく処理
        while True:
            # MAX_32BIT + 1 と１を足しているのは、range関数が０からカウントを始めるため。１を足すことで最大値であるMAX_32BITまで代入できる。
            for n in range(MAX_32BIT + 1):
                # Nonceを次々と更新
                block.nonce = n
                # Blockクラスで実装しているcheck_valid_hashの結果がTRUE、つまりターゲットより小さいハッシュ値が見つかれば、
                if block.check_valid_hash():
                    # 終了した時間と経過時間などを計算し、
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = \
                    str((end_time - start_time) / \
                        1000.0) + "秒"
                    # chainに追加
                    self.add_block(block)
                    # ブロックの情報を出力
                    self.getblockinfo()
                    return
            # ブロックのタイムスタンプと終了時のタイムスタンプが同じだった場合の処理を定義している。
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    # ジェネシスブロックを生成するためのメソッド
    # ジェネシスブロックはブロックチェーンにおける最初のブロックで他のブロックとは「prebhashの値を指定しないと参照する値がないという点」で異なっている。
    def create_genesis(self):
        # Blockクラスの引数にジェネシスブロックの情報を指定することで、genesis_blockインスタンスを生成。
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        # マイニングを行い、ジェネシスブロックの生成を行う。
        self.mining(genesis_block)

    # 最新のブロックの情報を引数として利用することで、次々と鎖上につなげていくことが可能
    def add_newblock(self, i):
        # ブロックチェーンの本体情報であるchainの配列の一番後ろのブロックをlast_blockとして取り出し、それを引数にBlockクラスをインスタンス化する
        last_block = self.chain[-1]
        # last_blockのblock_hashとbitsをそれぞれ引数として指定。
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    # ブロックチェーンクラスをインスタンス化
    bc = BlockChain(INITIAL_BITS)
    print("ジェネシスブロックを作成中 ・・・")
    # ジェネシスブロックを生成
    bc.create_genesis()
    # 30個の新規ブロックの生成
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中 ・・・
{
  "bits": "1d777777",
  "block_hash": "000000084f4933fbcf77b5f0b3100c799b482dc04d4fc0d06fced03889f266df",
  "elapsed_time": "145.336秒",
  "index": 0,
  "nonce": "1799053",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2022/06/14 11:06:43"
}
2番目のブロックを作成中・・・
